### Greedy Algorithms

In [5]:
class Item(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.weight = w
    def getName(self):
        return self.name
    def getValue(self):
        return self.value
    def getWeight(self):
        return self.weight
    def __str__(self):
        res = '<'+self.name+', '+str(self.value)+', '+str(self.weight)+'>'
        return res
    
def value(item):
    return item.getValue()
def weightInverse(item):
    return 1.0 / item.getWeight()
def density(item):
    return item.getValue()/item.getWeight()
def buildItems():
    names = ['clock','painting','radio','vase','book','computer']
    vals = [175,90,20,50,10,200]
    weights = [10,9,4,2,1,20]
    Items = []
    for i in range(len(vals)):
        Items.append(Item(names[i],vals[i],weights[i]))
    return Items
def greedy(Items, maxWeight, keyFcn):
    assert type(Items) == list and maxWeight >= 0
    ItemsCopy = sorted(Items, key=keyFcn, reverse=True)
    result = []
    totalVal = 0.0
    totalWeight = 0.0
    i = 0
    while totalWeight < maxWeight and i < len(Items):
        if(totalWeight+ItemsCopy[i].getWeight()) <= maxWeight:
            result.append(ItemsCopy[i])
            totalWeight += ItemsCopy[i].getWeight()
            totalVal += ItemsCopy[i].getValue()
        i += 1
    return (result,totalVal)
def testGreedy(Items, constraint, getKey):
    taken, val = greedy(Items, constraint, getKey)
    print('Total value of items taken = ', str(val))
    for item in taken:
        print(' ',item)

In [6]:
maxWeight = 20
Items = buildItems()
print('Use greedy by value for knapsack of size maxWeight')
testGreedy(Items, maxWeight, value)
print('Use greedy by weight to fill knapsack of size maxWeight')
testGreedy(Items,maxWeight,weightInverse)
print('Use greedy by density for knapsack of size maxWeight')
testGreedy(Items, maxWeight, density)

Use greedy by value for knapsack of size maxWeight
Total value of items taken =  200.0
  <computer, 200, 20>
Use greedy by weight to fill knapsack of size maxWeight
Total value of items taken =  170.0
  <book, 10, 1>
  <vase, 50, 2>
  <radio, 20, 4>
  <painting, 90, 9>
Use greedy by density for knapsack of size maxWeight
Total value of items taken =  255.0
  <vase, 50, 2>
  <clock, 175, 10>
  <book, 10, 1>
  <radio, 20, 4>


### Graph Theory

In [2]:
class Node(object):
    def __init__(self, name):
        self.name = str(name)
    def getName(self):
        return self.name
    def __str__(self):
        return self.name
    
class Edge(object):
    def __init__(self, src, dest):
        self.src = src
        self.dest = dest
    def getSource(self):
        return self.src
    def getDestination(self):
        return self.dest
    def __str__(self):
        return str(self.src) + '->' + str(self.dest)
    
class WeightedEdge(Edge):
    def __init__(self, src, dest, weight = 1.0):
        self.src = src
        self.dest = dest
        self.weight = weight
    def getWeight(self):
        return self.weight
    def __str__(self):
        return str(self.src)+'->('+str(self.weight)+')'+str(self.dest)

In [24]:
class Digraph(object):
    """edges is a dictionary mapping each node to a list of its children"""
    def __init__(self):
        self.nodes = []
        self.edges = {}
    def addNode(self, node):
        if node in self.nodes:
            raise ValueError('Duplicate node')
        else:
            self.nodes.append(node)
            self.edges[node] = []
            
    def getNode(self,name):
        for node in self.nodes:
            if node.getName() == name:
                return node
            
    def addEdge(self, edge):
        src = edge.getSource()
#         print(src in self.nodes)
        dest = edge.getDestination()
#         print(dest in self.nodes)
        if not(src in self.nodes and dest in self.nodes):
            raise ValueError('Node not in graph')
        self.edges[src].append(dest)
        
    def childrenOf(self, node):
        return self.edges[node]
    
    def hasNode(self, node):
        return node in self.nodes
    
    def __str__(self):
        res = ''
        for k in self.edges:
            for d in self.edges[k]:
                res += str(k)+'->'+str(d)+'\n'
        return res[:-1]

class Graph(Digraph):
    def addEdge(self, edge):
        Digraph.addEdge(self,edge)
        rev = Edge(edge.getDestination(),edge.getSource())
        Digraph.addEdge(self, rev)

In [26]:
# build the graph
g = Digraph()
cities = ['Boston','Providence','New York','Chicago','Denver','Phoenix','Los Angeles']

for city in cities:
    g.addNode(Node(city))

g.addEdge(Edge(g.getNode('Boston'),g.getNode('Providence')))    
g.addEdge(Edge(g.getNode('Boston'),g.getNode('New York')))
g.addEdge(Edge(g.getNode('Providence'),g.getNode('Boston')))
g.addEdge(Edge(g.getNode('Providence'),g.getNode('New York')))
g.addEdge(Edge(g.getNode('New York'),g.getNode('Chicago')))
g.addEdge(Edge(g.getNode('Chicago'),g.getNode('Denver')))
g.addEdge(Edge(g.getNode('Denver'),g.getNode('Phoenix')))
g.addEdge(Edge(g.getNode('Denver'),g.getNode('New York')))
g.addEdge(Edge(g.getNode('Chicago'),g.getNode('Phoenix')))
g.addEdge(Edge(g.getNode('Los Angeles'),g.getNode('Boston')))
print(g)

Chicago->Denver
Chicago->Phoenix
Denver->Phoenix
Denver->New York
Boston->Providence
Boston->New York
Providence->Boston
Providence->New York
Los Angeles->Boston
New York->Chicago
